# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2019 Semester 1
-----
## Project 1: Gaining Information about Naive Bayes
-----
###### Student Name(s): Simon Jerome Han
###### Python version: 3.7.1
###### Submission deadline: 1pm, Fri 5 Apr 2019

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [115]:
# Libraries that are useful
from collections import Counter
import numpy as np
import random

from math import ceil

In [52]:
# This function should open a data file in csv, and transform it into a usable format 
def preprocess(fileName, header=0):
    
    instances = []
    labels = []
    
    labelsUnique = []
    
    # read data using IO, since pandas seems to be discouraged
    with open(fileName, "r") as f:
        for line in f.readlines():
            
            lineData = line.strip().split(",")
            instance = lineData[:-1]
            label = lineData[-1]
            
            instances.append(instance)
            labels.append(label)
            
            if label not in labelsUnique:
                labelsUnique.append(label)
    
    return (instances, labels), labelsUnique
    
    

In [53]:
# This function should build a supervised NB model
def train(data):

    instances = data[0]
    labels = data[1]

    # Number of attributes
    numberOfAttributes = len(instances[0])
    numberOfInstances = len(instances)
  
    assert(numberOfInstances == len(labels))
    
    # unique list of labels
    labelsUnique = []
    labelCounts = Counter()
    for label in labels:
        labelCounts[label] += 1
        if label not in labelsUnique:
            labelsUnique.append(label)
        
    # Set up dictionary of posterior counts, and dictionary of posterior probabilities
    posteriorCounts = {}
    posteriorProbabilities = {}
    for label in labelsUnique:
        posteriorCounts[label] = []
        posteriorProbabilities[label] = []
        for i in range(0, numberOfAttributes):
            posteriorCounts[label].append(Counter())
            posteriorProbabilities[label].append({})
    
    # Get posterior counts from each instance in training data
    for i in range(0, numberOfInstances):
        instance = instances[i]
        label = labels[i]
        for j in range(0, numberOfAttributes):
            attributeValue = instance[j]
            if attributeValue not in ('?', ''):
                posteriorCounts[label][j][attributeValue] += 1

    # Get posterior probabilities
    for label in labelsUnique:
        attributeValueCounts = posteriorCounts[label]
        for i in range(0, numberOfAttributes):
            totalValueCount = sum(attributeValueCounts[i].values())
            for attributeValue in attributeValueCounts[i].keys():
                posteriorProbabilities[label][i][attributeValue] = attributeValueCounts[i][attributeValue] / totalValueCount
   
    # Get prior probabilities
    priorProbabilities = {}
    for label in labelsUnique:
        priorProbabilities[label] = labelCounts[label] / numberOfInstances
        
    return (priorProbabilities, posteriorProbabilities)

In [178]:
# This function should predict the class for an instance or a set of instances, based on a trained model 
def predict(model, instances, labelsUnique):
    
    priors = model[0]
    posteriors = model[1]
    predictions = []

    for instance in instances:
        # Keep track of label with the highest prediction value
        maxProb = -np.inf
        maxProbLabel = ""
        
        for label in labelsUnique:
            if label in priors.keys():
                logPrediction = np.log(priors[label])
            else:
                logPrediction = 0
            for i in range(0, len(instance)):
                if instance[i] not in ('?', '') and label in posteriors.keys():  
                    if posteriors[label][i].get(instance[i], 0) != 0:
                        logPrediction += np.log(posteriors[label][i].get(instance[i], 0))
            
            if logPrediction > maxProb:
                maxProb = logPrediction
                maxProbLabel = label  
        
        predictions.append(maxProbLabel)
        
    return predictions

In [179]:
# This function should evaluate a set of predictions, in a supervised context 
def evaluate(results, labelsUnique, micro = False, weighted = False):
    
    predictions = results[0]
    actual = results[1]
    
    assert(len(predictions) == len(actual))
    
    # Precision and recall, by class label
    precision = {}
    recall = {}
    
    # True positive, false negative and false positive counts for each class label
    tpLabel = {}
    fpLabel = {}
    fnLabel = {}
    
    # Get a simple measure of accuracy
    correct = 0
    for i in range(0, len(predictions)):
        if predictions[i] == actual[i]:
            correct += 1
    accuracy = correct/len(predictions)
    
    # Get precision and recall for each label
    for label in labelsUnique:
        tp = 0
        tn = 0
        fp = 0
        fn = 0
        for i in range(0, len(predictions)):
            if predictions[i] == label:
                if actual[i] == label:
                    tp += 1
                else:
                    fp += 1
            else:
                if actual[i] == label:
                    fn += 1
                else:
                    tn += 1
                    
        tpLabel[label] = tp
        fpLabel[label] = fp
        fnLabel[label] = fn
        
        if not (tp == 0 and (fp == 0 or fn == 0)):
            precision[label] = tp / (tp + fp)
            recall[label] = tp / (tp + fn)
        else:
            precision[label] = 0
            recall[label] = 0
    
    if not micro and not weighted:
        # Macro Average, unweighted 
        precisionMacro = sum(precision.values())/len(labelsUnique)
        recallMacro = sum(recall.values())/len(labelsUnique)
        return accuracy, precisionMacro, recallMacro
    
    elif not micro and weighted:
        # Weighted macro average
        
        # Get label counts
        labelCounts = Counter()
        for label in actual:
            labelCounts[label] += 1
        
        precisionMacroWeighted = sum([(labelCounts[label]/len(actual))*precision[label] for label in labelsUnique])
        recallMacroWeighted = sum([(labelCounts[label]/len(actual))*recall[label] for label in labelsUnique])
        return accuracy, precisionMacroWeighted, recallMacroWeighted
        
    else:
        # Micro Average
        precisionMicro = sum(tpLabel.values()) / (sum(tpLabel.values()) + sum(fpLabel.values()))
        recallMicro = sum(tpLabel.values()) / (sum(tpLabel.values()) + sum(fnLabel.values()))
        return accuracy, precisionMicro, recallMicro
    

In [180]:
# This function should calculate the Information Gain of an attribute or a set of attribute, with respect to the class
def info_gain(model, data, labelsUnique):
    
    priors = model[0]
    posteriors = model[1]
    instances = data[0]
    labels = data[1]
    
    classEntropy = -sum([(priors[label]*np.log2(priors[label])) for label in labelsUnique])

    numberOfAttributes = len(list(posteriors.values())[0])
    attributeCounts = []
    attributeLabelCounts = []
    for i in range(0, numberOfAttributes):
        attributeCounts.append(Counter())
        attributeLabelCounts.append({})
    
    # Get attribute counts and label counts
    for i in range(0, len(instances)):
        for j in range(0, numberOfAttributes):
            val = instances[i][j]
            label = labels[i]
            if val not in ('?', ''):
                attributeCounts[j][val] += 1
                attributeLabelCounts[j][val] = attributeLabelCounts[j].get(val, Counter())
                attributeLabelCounts[j][val][label] += 1
    
    attributeEntropies = []
    for i in range(0, numberOfAttributes):
        counts = attributeLabelCounts[i]
        attributeEntropies.append({})
        for (key, val) in counts.items():
            total = sum(val.values())
            entropy = -sum([(v/total)*(np.log2(v/total)) for v in val.values()])
            attributeEntropies[i][key] = entropy

    informationGain = []
    for i in range(0, numberOfAttributes):
        counts = attributeCounts[i]
        total = sum(counts.values())
        meanInfo = sum([(counts[key]/total)*attributeEntropies[i][key] for key in counts.keys()])
        gain = classEntropy - meanInfo
        informationGain.append(gain)
        
    return informationGain

In [181]:
def processDataSet(name):
    data, labelsUnique = preprocess("2019S1-proj1-data/{}.csv".format(name), header=None)
    model = train(data)
    prediction = predict(model, data[0], labelsUnique)
    results = (prediction, data[1])
    accuracy, precision, recall = evaluate(results, labelsUnique, micro=False, weighted=True)
    ig = info_gain(model, data, labelsUnique)
    print("accuracy: {}".format(accuracy))
    print("precision: {}, recall: {}".format(precision, recall))
    print("information gain for each attribute: \n{}".format(ig))

In [182]:
dataSetNames = ["anneal", "breast-cancer", "car", "cmc", "hepatitis", "hypothyroid", "mushroom", "nursery", "primary-tumor"]
for name in dataSetNames:
    print("Dataset: {}.csv".format(name))
    processDataSet(name)
    print("")

Dataset: anneal.csv
accuracy: 0.22271714922049
precision: 0.7738957788696961, recall: 0.22271714922048996
information gain for each attribute: 
[0.40908953764450984, 0.0, 0.30605153542894037, 0.051344088764403883, 0.29108220585994704, 0.14711886228095583, 0.21372288031590858, 0.29223544065798435, 0.1261663361036094, 0.14107379163812883, 0.03248840649184159, 0.43517783626288553, 0.03870173274881039, 0.0004376065202116308, 0.03935557414283686, 0.021775078259213432, 0.03799747881351134, 0.03670308136440825, 0.0, 0.11722522630372034, 0.029753745208638716, 0.02704235332867655, 0.0, 0.015604780443500443, 0.13718113252042552, 0.0, 0.02239708985164568, 0.01824168402125026, 0.0, 0.0, 0.0, 0.043239605565149386, 0.033037571177056746, 0.019378864328319256, 0.003958783545891853]

Dataset: breast-cancer.csv
accuracy: 0.7377622377622378
precision: 0.7351624344092063, recall: 0.7377622377622378
information gain for each attribute: 
[0.010605956535614136, 0.0020016149737116518, 0.05717112532429669, 0.0

Questions (you may respond in a cell or cells below):

1. The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.
2. The Information Gain can be seen as a kind of correlation coefficient between a pair of attributes: when the gain is low, the attribute values are uncorrelated; when the gain is high, the attribute values are correlated. In supervised ML, we typically calculate the Infomation Gain between a single attribute and the class, but it can be calculated for any pair of attributes. Using the pair-wise IG as a proxy for attribute interdependence, in which cases are our NB assumptions violated? Describe any evidence (or indeed, lack of evidence) that this is has some effect on the effectiveness of the NB classifier.
3. Since we have gone to all of the effort of calculating Infomation Gain, we might as well use that as a criterion for building a “Decision Stump” (1-R classifier). How does the effectiveness of this classifier compare to Naive Bayes? Identify one or more cases where the effectiveness is notably different, and explain why.
4. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)
5. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.
6. Naive Bayes is said to elegantly handle missing attribute values. For the datasets with missing values, is there any evidence that the performance is different on the instances with missing values, compared to the instances where all of the values are present? Does it matter which, or how many values are missing? Would a imputation strategy have any effect on this?

Don't forget that groups of 1 student should respond to question (1), and one other question of your choosing. Groups of 2 students should respond to question (1) and question (2), and two other questions of your choosing. Your responses should be about 150-250 words each.

## Question 1
The Naive Bayes classifiers can be seen to vary, in terms of their effectiveness on the given datasets (e.g. in terms of Accuracy). Consider the Information Gain of each attribute, relative to the class distribution — does this help to explain the classifiers’ behaviour? Identify any results that are particularly surprising, and explain why they occur.

#### Answer
Hi

## Question 4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy. How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

#### Solution: Implementation

In [183]:
def NB_TrainEvaluate_holdout(testPercentage, dataFileName):
    
    data, labelsUnique = preprocess("2019S1-proj1-data/{}.csv".format(dataFileName), header=None)
    attributes = data[0]
    labels = data[1]
    
    attributeLabelPairs = [(attributes[i], labels[i]) for i in range(0, len(attributes))]
    random.shuffle(attributeLabelPairs)
    
    numTest = int(testPercentage*len(attributes))
    if numTest < 1:
        numTest = 1
    elif numTest == len(attributes):
        numTest -= 1
    testData = attributeLabelPairs[0:numTest]
    trainData = attributeLabelPairs[numTest:]
    
    testAttributes = [x[0] for x in testData]
    testLabels = [x[1] for x in testData]
    trainAttributes = [x[0] for x in trainData]
    trainLabels = [x[1] for x in trainData]
    
    model = train((trainAttributes, trainLabels))
    prediction = predict(model, testAttributes, labelsUnique)
    results = (prediction, testLabels)
    accuracy, precision, recall = evaluate(results, labelsUnique, micro=False, weighted=True)
    
    return accuracy, precision, recall

In [187]:
def NB_RepeatedRandomSubsampling(testPercentage, iters, dataFileName):
    cumulatedAccuracy = 0
    cumulatedPrecision = 0
    cumulatedRecall = 0
    for i in range(iters):
        accuracy, precision, recall = NB_TrainEvaluate_holdout(testPercentage, dataFileName)
        cumulatedAccuracy += accuracy
        cumulatedPrecision += precision
        cumulatedRecall += recall
    return cumulatedAccuracy/iters, cumulatedPrecision/iters, cumulatedRecall/iters

In [188]:
def NB_TrainEvaluate_CrossValidation(m, dataFileName):
    data, labelsUnique = preprocess("2019S1-proj1-data/{}.csv".format(dataFileName), header=None)
    attributes = data[0]
    labels = data[1]
    
    attributeLabelPairs = [(attributes[i], labels[i]) for i in range(0, len(attributes))]
    random.shuffle(attributeLabelPairs)
    
    partitions = []
    partitionSize = ceil(len(attributes)/m)
    bottom = 0
    top = partitionSize
    for i in range(m):
        partitions.append(attributeLabelPairs[bottom:top])
        bottom += partitionSize
        top += partitionSize
        if top > len(attributes):
            top = len(attributes)
    
    totalAccuracy = 0
    totalPrecision = 0
    totalRecall = 0
    for i in range(m):
        testSet = partitions[i]
        trainSet = [y for x in [partitions[j] for j in range(m) if j != i] for y in x]
        
        testAttributes = [x[0] for x in testSet]
        testLabels = [x[1] for x in testSet]
        trainAttributes = [x[0] for x in trainSet]
        trainLabels = [x[1] for x in trainSet]
        
        model = train((trainAttributes, trainLabels))
        prediction = predict(model, testAttributes, labelsUnique)
        results = (prediction, testLabels)
        accuracy, precision, recall = evaluate(results, labelsUnique, micro=False, weighted=True)
        
        totalAccuracy += accuracy
        totalPrecision += precision
        totalRecall += recall
    
    return totalAccuracy/m, totalPrecision/m, totalRecall/m
        

In [189]:
def NB_TrainEvaluate_Naive(dataFileName):
    data, labelsUnique = preprocess("2019S1-proj1-data/{}.csv".format(dataFileName), header=None)
    model = train(data)
    prediction = predict(model, data[0], labelsUnique)
    results = (prediction, data[1])
    return evaluate(results, labelsUnique, micro=False, weighted=True)
    

In [322]:
def compareEvaluationStrategies(dataSetName):
    name = dataSetName
    print("comparing evaluation strategies on the {}.csv dataset".format(name))
    print("                                    accuracy - precision - recall")
    print("Holdout (1 iteration):             {}".format(NB_TrainEvaluate_holdout(0.2, name)))
    print("Holdout with RRS ({} iterations): {}".format(100, NB_RepeatedRandomSubsampling(0.2, 100, name)))
    print("Cross Validation:                  {}".format(NB_TrainEvaluate_CrossValidation(10, name)))
    print("Train/Test on full dataset:        {}".format(NB_TrainEvaluate_Naive(name)))

In [323]:
for name in dataSetNames:
    compareEvaluationStrategies(name)
    print("")

comparing evaluation strategies on the anneal.csv dataset
                                    accuracy - precision - recall
Holdout (1 iteration):             (0.16759776536312848, 0.7941706757086464, 0.16759776536312848)
Holdout with RRS (100 iterations): (0.18737430167597768, 0.7720212195801538, 0.18737430167597768)
Cross Validation:                  (0.20063131313131316, 0.7977885483345755, 0.2006313131313131)
Train/Test on full dataset:        (0.22271714922049, 0.7738957788696961, 0.22271714922048996)

comparing evaluation strategies on the breast-cancer.csv dataset
                                    accuracy - precision - recall
Holdout (1 iteration):             (0.7192982456140351, 0.7192982456140351, 0.7192982456140351)
Holdout with RRS (100 iterations): (0.7284210526315792, 0.7347288447702209, 0.7284210526315792)
Cross Validation:                  (0.7311724137931034, 0.7656376250243527, 0.7311724137931034)
Train/Test on full dataset:        (0.7377622377622378, 0.7351624344

#### Solution: Why isn't there much of a difference between the evaluation results?
Hi

## Question 5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the Naive Bayes classifier? Explain why, or why not.

#### Solution: Implementation

In [318]:
def trainWithSpecialSmoothing(data, addK=False, k=0):

    instances = data[0]
    labels = data[1]

    # Number of attributes
    numberOfAttributes = len(instances[0])
    numberOfInstances = len(instances)
  
    assert(numberOfInstances == len(labels))
    
    # unique list of labels
    labelsUnique = []
    labelCounts = Counter()
    for label in labels:
        labelCounts[label] += 1
        if label not in labelsUnique:
            labelsUnique.append(label)
        
    # Set up dictionary of posterior counts, and dictionary of posterior probabilities
    posteriorCounts = {}
    posteriorProbabilities = {}
    for label in labelsUnique:
        posteriorCounts[label] = []
        posteriorProbabilities[label] = []
        for i in range(0, numberOfAttributes):
            posteriorCounts[label].append(Counter())
            posteriorProbabilities[label].append({})
            
    # List of attribute values for each attribute
    attributesUnique = []
    for i in range(0, numberOfAttributes):
        attributesUnique.append([])
    
    # Get posterior counts from each instance in training data
    for i in range(0, numberOfInstances):
        instance = instances[i]
        label = labels[i]
        for j in range(0, numberOfAttributes):
            attributeValue = instance[j]
            if attributeValue not in ('?', ''):
                posteriorCounts[label][j][attributeValue] += 1
                if attributeValue not in attributesUnique[j]:
                    attributesUnique[j].append(attributeValue)

    # Get posterior probabilities
    for label in labelsUnique:
        attributeValueCounts = posteriorCounts[label]
        for i in range(0, numberOfAttributes):
            totalValueCount = sum(attributeValueCounts[i].values())
            for attributeValue in attributeValueCounts[i].keys():
                if addK:
                    posteriorProbabilities[label][i][attributeValue] = (k + attributeValueCounts[i][attributeValue])/(len(attributesUnique[i]) + labelCounts[label])
                else:
                    posteriorProbabilities[label][i][attributeValue] = attributeValueCounts[i][attributeValue] / totalValueCount
   
    # Get prior probabilities
    priorProbabilities = {}
    for label in labelsUnique:
        priorProbabilities[label] = labelCounts[label] / numberOfInstances
        
    return (priorProbabilities, posteriorProbabilities), (labelCounts, [len(x) for x in attributesUnique])

In [319]:
def predictWithSpecialSmoothing(model, instances, labelsUnique, counts, addK=False, k=0):
    
    priors = model[0]
    posteriors = model[1]
    predictions = []
    
    labelCounts = counts[0]
    attributeCounts = counts[1]

    for instance in instances:
        # Keep track of label with the highest prediction value
        maxProb = -np.inf
        maxProbLabel = ""
        
        for label in labelsUnique:
            if label in priors.keys():
                logPrediction = np.log(priors[label])
            else:
                logPrediction = 0
            for i in range(0, len(instance)):
                if instance[i] not in ('?', '') and label in posteriors.keys(): 
                    if posteriors[label][i].get(instance[i], 0) != 0:
                        logPrediction += np.log(posteriors[label][i].get(instance[i], 0))
                    elif addK:
                        logPrediction += np.log(k/(attributeCounts[i] + labelCounts[label]))
            
            if logPrediction > maxProb:
                maxProb = logPrediction
                maxProbLabel = label  
        
        predictions.append(maxProbLabel)
        
    return predictions

In [320]:
def NB_TrainEvaluate_SpecialSmoothing_CrossValidation(dataFileName, m=10, addK=False, k=0):
    data, labelsUnique = preprocess("2019S1-proj1-data/{}.csv".format(dataFileName), header=None)
    attributes = data[0]
    labels = data[1]

    attributeLabelPairs = [(attributes[i], labels[i]) for i in range(0, len(attributes))]
    random.shuffle(attributeLabelPairs)

    partitions = []
    partitionSize = ceil(len(attributes)/m)
    bottom = 0
    top = partitionSize
    for i in range(m):
        partitions.append(attributeLabelPairs[bottom:top])
        bottom += partitionSize
        top += partitionSize
        if top > len(attributes):
            top = len(attributes)

    totalAccuracy = 0
    totalPrecision = 0
    totalRecall = 0
    for i in range(m):
        testSet = partitions[i]
        trainSet = [y for x in [partitions[j] for j in range(m) if j != i] for y in x]

        testAttributes = [x[0] for x in testSet]
        testLabels = [x[1] for x in testSet]
        trainAttributes = [x[0] for x in trainSet]
        trainLabels = [x[1] for x in trainSet]
  
        model, counts = trainWithSpecialSmoothing((trainAttributes, trainLabels), addK=addK, k=k)
        prediction = predictWithSpecialSmoothing(model, testAttributes, labelsUnique, counts, addK=addK, k=k)
        results = (prediction, testLabels)
        accuracy, precision, recall = evaluate(results, labelsUnique, micro=False, weighted=True)

        totalAccuracy += accuracy
        totalPrecision += precision
        totalRecall += recall

    return totalAccuracy/m, totalPrecision/m, totalRecall/m

In [321]:
for name in dataSetNames:
    addK_accuracy, addK_precision, addK_recall = NB_TrainEvaluate_SpecialSmoothing_CrossValidation(name, m=10, addK=True, k=1)
    accuracy, precision, recall = NB_TrainEvaluate_SpecialSmoothing_CrossValidation(name, m=10, addK=False)
    print("dataset: {}.csv".format(name))
    print("no smoothing:          Accuracy: {}   Precision: {}   Recall: {}".format(accuracy, precision, recall))
    print("using addK smoothing:  Accuracy: {}   Precision: {}   Recall: {}".format(addK_accuracy, addK_precision, addK_recall))
    print("")

dataset: anneal.csv
no smoothing:          Accuracy: 0.20287878787878788   Precision: 0.772739264965789   Recall: 0.20287878787878783
using addK smoothing:  Accuracy: 0.9198484848484849   Precision: 0.9579714303454976   Recall: 0.9198484848484849

dataset: breast-cancer.csv
no smoothing:          Accuracy: 0.7190344827586206   Precision: 0.7280325447166527   Recall: 0.7190344827586206
using addK smoothing:  Accuracy: 0.7295172413793103   Precision: 0.7381726184526785   Recall: 0.7295172413793103

dataset: car.csv
no smoothing:          Accuracy: 0.04515431159787715   Precision: 0.33169744252339195   Recall: 0.04515431159787715
using addK smoothing:  Accuracy: 0.8506980360342087   Precision: 0.849522983570585   Recall: 0.8506980360342089

dataset: cmc.csv
no smoothing:          Accuracy: 0.4530237684493004   Precision: 0.478056902869768   Recall: 0.4530237684493004
using addK smoothing:  Accuracy: 0.49856718420548213   Precision: 0.5152636881163952   Recall: 0.49856718420548213

dataset